In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.FilteredSkeleton.describe()
#ta3p100.FilteredSkeletonMinusSoma.describe()
#ta3p100.FilteredSkeletonMinusSoma.drop()


In [4]:
#function that takes in a 3x3 array of coordinates for faces and returns triangles and vertices
def index_unique_rows(full_coordinate_array):
    """
    Separates an array of nested coordinate rows into an array of unique rows and and index array.
    """
    vertices, flat_idx = np.unique(full_coordinate_array.reshape(-1, full_coordinate_array.shape[-1]), axis=0, return_inverse=True)
    return vertices, flat_idx.reshape(-1, full_coordinate_array.shape[-2])





In [5]:
#will take in and populate the soma table based on the key it gets
def soma_verts_faces(query_key):
    
    table=""
    vertices_soma,triangles_soma = (ta3p100.CompartmentFinal.ComponentFinal() & query_key
                                    & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")

    if len(vertices_soma) > 0:
        print("Soma found in Exhitatory")
        #get the regular mesh from CleansedMesh
        vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")
    else:
        vertices_soma,triangles_soma = (ta3p100.CompartmentOrphan.ComponentOrphan() & query_key & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")
        if len(vertices_soma) > 0:
            print("Soma found in Orphans")
            vertices_mesh,triangles_mesh = (ta3p100.CleansedMeshOrphan & query_key).fetch("vertices","triangles")
        else:
            print("No Soma exists for " + str(query_key["segment_id"]))
            return np.array([]),np.array([])
            
    ts_flatten = np.hstack(triangles_soma).astype("int64")

    vertices_real = vertices_mesh[0]
    triangles_real = triangles_mesh[0]

    ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

    vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
    return vertices_whole, triangles_whole

In [6]:
def filter_edges_by_bounding_box(edges,max_bb_zone,min_bb_zone):
    #can just use bounding box function to get rid of any inside edges
    filtered_remaining = list()

    for i,e in enumerate(edges):
        #print(e)
        if min(e[0][0],e[1][0])>max_bb_zone[0]:
            #print("minx>maxx")
            filtered_remaining.append(e)
            
            continue

        if max(e[0][0],e[1][0])<min_bb_zone[0]:
            #print("maxx<minx")
            filtered_remaining.append(e)
            continue

        if min(e[0][1],e[1][1])>max_bb_zone[1]:
            #print("miny>maxy")
            filtered_remaining.append(e)
            continue

        if max(e[0][1],e[1][1])<min_bb_zone[1]:
            #print("maxy<miny")
            filtered_remaining.append(e)
            continue

        if min(e[0][2],e[1][2])>max_bb_zone[2]:
            #print("minz>maxz")
            filtered_remaining.append(e)
            continue

        if max(e[0][2],e[1][2])<min_bb_zone[2]:
            #print("maxz<minz")
            filtered_remaining.append(e)
            continue

        #filtered_edges.append(e)

    return np.array(filtered_remaining)

In [7]:
exhitatory_with_somas = dj.U("segment_id","segmentation") & ((ta3p100.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'").proj("segment_id"))
#exhitatory_with_somas

In [8]:
orphans_with_somas = dj.U("segment_id","segmentation") & ((ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Soma'").proj("segment_id"))
#orphans_with_somas


In [9]:
total_soma = ((dj.U("segment_id","segmentation") & ((ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Soma'").proj("segment_id")).proj()) + 
(dj.U("segment_id","segmentation") & ((ta3p100.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'").proj("segment_id"))
.proj()))
    
#total_soma



In [10]:
#(ta3p100.FilteredSkeleton & total_soma)

In [11]:
total_neurons_with_somas = exhitatory_with_somas.proj() + orphans_with_somas.proj()
print((len(total_neurons_with_somas),35 +318))


(353, 353)


In [12]:
@schema
class FilteredSkeletonMinusSoma(dj.Computed):
    definition="""
    -> ta3p100.FilteredSkeleton
    ---
    vertices              :longblob #vertex coordinates of soma mesh
    triangles             :longblob #faces for soma mesh
    edges                  :longblob #edges of skeelton after soma edges removed
    n_edges               :int unsigned #number of edges of skeelton after soma edges removed
    soma_bounding_corners :blob #bounding box corners for the soma mesh
    """
    
    key_source = ta3p100.FilteredSkeleton & ((dj.U("segment_id","segmentation") & ((ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Soma'").proj("segment_id")).proj()) + 
                (dj.U("segment_id","segmentation") & ((ta3p100.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'").proj("segment_id")).proj()))
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
        print()
        print()
        print(str(key["segment_id"])+ ":")
        global_start_time = time.time()
        #create return key
        return_key = key.copy()

        #pull down the skeleton for the mesh
        skeleton_data = (ta3p100.FilteredSkeleton() & key).fetch(as_dict=True)[0]
        
        #get the vertices and triangles for the Soma
        start_time = time.time()
        vertices_whole, triangles_whole = soma_verts_faces(key)
        print(f"Step 1: extracted Soma Mesh = {time.time()-start_time}")

        #if no soma portion was found then just write regular skeleton
        if not vertices_whole.any():
            
            print("No Soma Found")

            #return_key["soma_exist"] = False
            return_key["vertices"] = vertices_whole
            return_key["triangles"] = triangles_whole
            return_key["edges"] = skeleton_data["edges"]
            return_key["n_edges"] = skeleton_data["n_edges"]
            return_key["soma_bounding_corners"] = np.array([])

            self.insert(return_key,skip_duplicates=True)

        #just need to strip the portions of the skeleton that are inside of the mesh

        #find the bounding box
        start_time = time.time()
        mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
        min_bb = np.array(mesh.bounding_box.vertices).min(0)
        max_bb = np.array(mesh.bounding_box.vertices).max(0)
        print(f"Step 2: Calculated Bounding Box = {time.time()-start_time}")

        start_time = time.time()
        #get the filtered edges according to bounding box:
        filtered_edges_postsyn = filter_edges_by_bounding_box(skeleton_data["edges"],max_bb,min_bb)

        print(f"Step 3: filtering edges = {time.time()-start_time}")

        

        #write off the new data to the table
        #return_key["soma_exist"] = True
        
        start_time = time.time()
        
        return_key["vertices"] = vertices_whole
        return_key["triangles"] = triangles_whole
        return_key["edges"] = filtered_edges_postsyn
        return_key["n_edges"] = filtered_edges_postsyn.shape[0]
        return_key["soma_bounding_corners"] = np.array((min_bb,max_bb))
        
#         print(return_key)
#         print(return_key.keys())
#         print("len(return_key.keys()) = " + str(len(return_key.keys())))
        
#         for k in return_key.keys():
#             print("type(return_key["+k+"])=" + str(type(return_key[k])))
        
        self.insert1(return_key,skip_duplicates=True,ignore_extra_fields=True)
        print(f"Step 4: Inserted Key = {time.time()-start_time}")
        print(f"Total time = {time.time()-global_start_time}")
    
    

In [13]:
#(schema.jobs & "table_name='__filtered_skeleton_minus_soma'").delete()

In [14]:
#FilteredSkeletonMinusSoma.drop()

In [15]:
notebook_start_time = time.time()
FilteredSkeletonMinusSoma.populate(reserve_jobs=True)
print(f"Entire Notebook = {time.time()-notebook_start_time}")



648518346341353574:
Soma found in Orphans
Step 1: extracted Soma Mesh = 0.16516637802124023
Step 2: Calculated Bounding Box = 0.016474246978759766
Step 3: filtering edges = 0.003059864044189453
Step 4: Inserted Key = 0.09164905548095703
Total time = 0.28789615631103516


648518346341369721:
Soma found in Orphans
Step 1: extracted Soma Mesh = 0.09931373596191406
Step 2: Calculated Bounding Box = 0.012188911437988281
Step 3: filtering edges = 0.003915548324584961
Step 4: Inserted Key = 0.05376553535461426
Total time = 0.18009710311889648


648518346341373109:
Soma found in Orphans
Step 1: extracted Soma Mesh = 0.07219696044921875
Step 2: Calculated Bounding Box = 0.009088277816772461
Step 3: filtering edges = 0.003245115280151367
Step 4: Inserted Key = 0.033730506896972656
Total time = 0.12697529792785645


648518346341375427:
Soma found in Orphans
Step 1: extracted Soma Mesh = 0.2788379192352295
Step 2: Calculated Bounding Box = 0.016603469848632812
Step 3: filtering edges = 0.0061287

In [19]:
ta3p100.CleansedMeshOrphan & (FilteredSkeletonMinusSoma() & "n_edges=0").proj()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices,n_triangles,vertices,triangles
2,648518346341376439,0.35,5636,11234,=BLOB=,=BLOB=
2,648518346343271774,0.35,3078,6124,=BLOB=,=BLOB=
2,648518346343462325,0.35,4692,9340,=BLOB=,=BLOB=
2,648518346343784612,0.35,15065,29972,=BLOB=,=BLOB=


In [17]:
#investigating the error: arrays used as indices must be of integer (or boolean) type
query_key = dict(segmentation=2,segment_id=648518346341378749)

table=""
vertices_soma,triangles_soma = (ta3p100.CompartmentFinal.ComponentFinal() & query_key
                                & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")

if len(vertices_soma) > 0:
    print("Soma found in Exhitatory")
    #get the regular mesh from CleansedMesh
    vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")
else:
    vertices_soma,triangles_soma = (ta3p100.CompartmentOrphan.ComponentOrphan() & query_key).fetch("vertex_indices","triangle_indices")
    if len(vertices_soma) > 0:
        print("Soma found in Orphans")
        vertices_mesh,triangles_mesh = (ta3p100.CleansedMeshOrphan & query_key).fetch("vertices","triangles")
    else:
        print("No Soma exists for " + str(query_key["segment_id"]))
        #return np.array([]),np.array([])

ts_flatten = np.hstack(triangles_soma).astype("int64")
type(ts_flatten[0])

vertices_real = vertices_mesh[0]
triangles_real = triangles_mesh[0]

ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
#return vertices_whole, triangles_whole

Soma found in Orphans
